In [59]:
import requests
from requests.exceptions import HTTPError
import json
import pandas as pd


In [60]:
URL = "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json"
deliveries = None

try:
    resposta = requests.get(URL)
    resposta.raise_for_status()
except HTTPError as exc:
    print(exc)
else:
    deliveries = json.loads(resposta.text)
    

In [61]:
deliveries_df = pd.DataFrame(deliveries)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [62]:
origin_normalize_df = pd.concat([
    pd.DataFrame(deliveries_df['origin'].apply(lambda value: value['lng'])).rename(columns={'origin': 'origin_lng'}),
    pd.DataFrame(deliveries_df['origin'].apply(lambda value: value['lat'])).rename(columns={'origin': 'origin_lat'}),
    pd.DataFrame(deliveries_df['origin'].apply(lambda value: f'{value['lng']} {value['lat']}')),
], axis=1)

origin_normalize_df.head()

,origin_lng,origin_lat,origin
0,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
1,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
3,-47.893662,-15.805118,-47.89366206897872 -15.80511751066334
4,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274


In [63]:
deliveries_df = deliveries_df.drop('origin', axis=1)
deliveries_df = pd.merge(left= deliveries_df, right=origin_normalize_df, how='right', left_index=True, right_index=True )
deliveries_df.head()

,name,region,vehicle_capacity,deliveries,origin_lng,origin_lat,origin
0,cvrp-2-df-33,df-2,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
1,cvrp-2-df-73,df-2,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
2,cvrp-2-df-20,df-2,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145,-48.05498915846707 -15.83814451122274
3,cvrp-1-df-71,df-1,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118,-47.89366206897872 -15.80511751066334
4,cvrp-2-df-87,df-2,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145,-48.05498915846707 -15.83814451122274


In [64]:
destiny_exploded_df = deliveries_df[['deliveries']].explode('deliveries')
destiny_exploded_df.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [65]:
destiny_normalize_df = pd.concat([
    pd.DataFrame(destiny_exploded_df['deliveries'].apply(lambda value: value['point']['lng'])).rename(columns={'deliveries': 'destiny_lng'}),
    pd.DataFrame(destiny_exploded_df['deliveries'].apply(lambda value: value['point']['lat'])).rename(columns={'deliveries': 'destiny_lat'}),
    pd.DataFrame(destiny_exploded_df['deliveries'].apply(lambda value: f'{value['point']['lng']} {value['point']['lat']}')).rename(columns={'deliveries': 'destiny'}),
    pd.DataFrame(destiny_exploded_df['deliveries'].apply(lambda value: value['size'])).rename(columns={'deliveries': 'delivery_size'}),
], axis=1)

destiny_normalize_df.head()

,destiny_lng,destiny_lat,destiny,delivery_size
0,-48.116189,-15.848929,-48.11618888384239 -15.848929154862294,9
0,-48.118195,-15.850772,-48.11819489551 -15.850772371049631,2
0,-48.112483,-15.847871,-48.11248339849675 -15.84787055941764,1
0,-48.118023,-15.846471,-48.11802268617869 -15.846471025281456,2
0,-48.114898,-15.858055,-48.114898174591026 -15.85805462185708,7


In [66]:
deliveries_df = deliveries_df.drop('deliveries', axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=destiny_normalize_df, how='right', left_index=True, right_index=True)
deliveries_df.head()

,name,region,vehicle_capacity,origin_lng,origin_lat,origin,destiny_lng,destiny_lat,destiny,delivery_size
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,-48.116189,-15.848929,-48.11618888384239 -15.848929154862294,9
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,-48.118195,-15.850772,-48.11819489551 -15.850772371049631,2
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,-48.112483,-15.847871,-48.11248339849675 -15.84787055941764,1
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,-48.118023,-15.846471,-48.11802268617869 -15.846471025281456,2
0,cvrp-2-df-33,df-2,180,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,-48.114898,-15.858055,-48.114898174591026 -15.85805462185708,7


In [67]:
deliveries_df = deliveries_df[['name', 'region', 'origin_lng', 'origin_lat', 'origin', 'vehicle_capacity', 'delivery_size', 'destiny_lng', 'destiny_lat', 'destiny']]
deliveries_df.head()

,name,region,origin_lng,origin_lat,origin,vehicle_capacity,delivery_size,destiny_lng,destiny_lat,destiny
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,180,9,-48.116189,-15.848929,-48.11618888384239 -15.848929154862294
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,180,2,-48.118195,-15.850772,-48.11819489551 -15.850772371049631
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,180,1,-48.112483,-15.847871,-48.11248339849675 -15.84787055941764
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,180,2,-48.118023,-15.846471,-48.11802268617869 -15.846471025281456
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,-48.05498915846707 -15.83814451122274,180,7,-48.114898,-15.858055,-48.114898174591026 -15.85805462185708
